# 6.5 동적시각화 실습 Candlcstick 주식 차트 만들기

In [1]:
# 필요 라이브러리 로딩
import plotly.graph_objects as go # graph_objects 패키지 사용

import pandas as pd
import numpy as np

In [2]:
# 데이터 로딩 없으니 생성
# 날짜 생성
date = pd.DataFrame(pd.date_range('2020-01-02','2021-12-30',freq='B'))
date

,0
0,2020-01-02
1,2020-01-03
2,2020-01-06
3,2020-01-07
4,2020-01-08
...,...
516,2021-12-24
517,2021-12-27
518,2021-12-28
519,2021-12-29


In [370]:
# 화이트 노이즈
np.random.seed(1)
w_n = np.sin(np.linspace(0,50,521*4)*np.pi)*0.3
trend = (np.sin(np.linspace(1,3.4,521*4)*np.pi)*7 + np.random.randn(521*4) +np.linspace(282,320,521*4)+
         np.sin(np.linspace(0,11,521*4)*np.pi)*2 +np.sin(np.linspace(1,17,521*4)*np.pi)*3 + np.sin(np.linspace(1.7,6,521*4)*np.pi))
# price 생성
price = w_n + trend

#volume 생성
np.random.seed(2)
vol = pd.DataFrame(np.random.randint(1500000,1700000,size=521)+(np.sin(np.linspace(1,3.4,521)*np.pi)*500000)+ np.sin(np.linspace(1,15,521)*np.pi)*1000000)

In [371]:
data = {
    'High' : [],
    'Low' : [],
    'Open': [],
    'Close' : []}
for i in range(521):
    d_price = [price[i*4],price[i*4+1],price[i*4+1],price[i*4+1]]
    data['High'].append(round(max(d_price),1))
    data['Low'].append(round(min(d_price),1))
    data['Open'].append(round(d_price[0],1))
    data['Close'].append(round(d_price[3],1))
    
p_df = pd.DataFrame(data)

df = pd.merge(date, p_df, left_index=True, right_index=True, how='inner')
df = pd.merge(df, vol, left_index=True, right_index=True, how='inner')
df.columns = ['Date', 'High', 'Low', 'Open', 'Close', 'Volume']
df

,Date,High,Low,Open,Close,Volume
0,2020-01-02,282.8,280.6,282.8,280.6,1.589256e+06
1,2020-01-03,282.0,278.8,282.0,278.8,1.509149e+06
2,2020-01-06,281.3,280.8,281.3,280.8,1.412961e+06
3,2020-01-07,280.6,280.5,280.6,280.5,1.402866e+06
4,2020-01-08,280.6,279.9,280.6,279.9,1.301200e+06
...,...,...,...,...,...,...
516,2021-12-24,314.4,313.4,314.4,313.4,1.466620e+06
517,2021-12-27,315.6,314.9,314.9,315.6,1.366349e+06
518,2021-12-28,314.3,314.1,314.1,314.3,1.315531e+06
519,2021-12-29,314.4,314.0,314.4,314.0,1.256979e+06


In [372]:
# Date를인덱스로
df= df.set_index('Date', drop=True)
# 시계열데이터를 사용할땐 시계열을 pd.to_datetimeindex로 변환
df.index= pd.to_datetime(df.index)
df

,High,Low,Open,Close,Volume
Date,,,,,
2020-01-02,282.8,280.6,282.8,280.6,1.589256e+06
2020-01-03,282.0,278.8,282.0,278.8,1.509149e+06
2020-01-06,281.3,280.8,281.3,280.8,1.412961e+06
2020-01-07,280.6,280.5,280.6,280.5,1.402866e+06
2020-01-08,280.6,279.9,280.6,279.9,1.301200e+06
...,...,...,...,...,...
2021-12-24,314.4,313.4,314.4,313.4,1.466620e+06
2021-12-27,315.6,314.9,314.9,315.6,1.366349e+06
2021-12-28,314.3,314.1,314.1,314.3,1.315531e+06


**plotly**는 layout 설정이 조금은 복잡합니다. 특히 **layout**값을 입력할 때 항상 딕셔너리 구조로 넣어줘야하는 것이 특징입니다.
따라서 수정하고자 하는 항목이 많다면 중첩 딕셔너리를 사용해야 하는 경우가 있습니다(제목의 글자, 폰트 등을 바꾸려면 중첩 딕셔너리 사용)

### plotly의 graph_objects
**graph_objects**는 Figure 객체 데이터를 추가하는 것이 특징이며, **update_layout**등과 같은 메서드로 꾸미는 것이 특징입니다.

In [373]:
# Figure 객체 생성
fig = go.Figure()

In [374]:
type(fig)

plotly.graph_objs._figure.Figure

In [375]:
# fig 출력
print(fig)

Figure({
    'data': [], 'layout': {'template': '...'}
})


In [376]:
fig

### add_trace : 캔들스틱 차트 구현

In [377]:
fig.add_trace(
    go.Candlestick(
        x=df.index, # x축 날짜
        open=df.Open, # 시초가
        high=df.High, # 고가
        low=df.Low, # 저가
        close=df.Close, # 종가
        increasing_line_color='red', # 상승은 빨강
        decreasing_line_color='blue' # 하락은 파랑
    ))

### update_layout : 그래프 레이아웃 설정

In [378]:
print(fig)

Figure({
    'data': [{'close': array([280.6, 278.8, 280.8, ..., 314.3, 314. , 314.7]),
              'decreasing': {'line': {'color': 'blue'}},
              'high': array([282.8, 282. , 281.3, ..., 314.3, 314.4, 314.7]),
              'increasing': {'line': {'color': 'red'}},
              'low': array([280.6, 278.8, 280.8, ..., 314.1, 314. , 312. ]),
              'open': array([282.8, 282. , 281.3, ..., 314.1, 314.4, 312. ]),
              'type': 'candlestick',
              'x': array([datetime.datetime(2020, 1, 2, 0, 0),
                          datetime.datetime(2020, 1, 3, 0, 0),
                          datetime.datetime(2020, 1, 6, 0, 0), ...,
                          datetime.datetime(2021, 12, 28, 0, 0),
                          datetime.datetime(2021, 12, 29, 0, 0),
                          datetime.datetime(2021, 12, 30, 0, 0)], dtype=object)}],
    'layout': {'template': '...'}
})


In [379]:
# Figure 객체를 출력해보면 Figure({'data':[],'layout':{'template':'...'}}) 이런 구조의 속성값이 나온 것을 확인 할수 있었습니다.
# Figure 객체를 최초 생성시 data에 대한 값을 주는 방법으로 캔들스틱을 구현할 수 있습니다.
# 'ploty'의 'layout'은 다양하기 보다 간편하게 'update_layout'메서드로 업데이트 하겠습니다.

fig = go.Figure(
    data=[go.Candlestick(
        x=df.index, # x축 날짜
        open=df.Open, # 시초가
        high=df.High, # 고가
        low=df.Low, # 저가
        close=df.Close, # 종가
        increasing_line_color='red', # 상승은 빨강
        decreasing_line_color='blue' # 하락은 파랑
    )]
)

fig.show()

In [380]:
# graph_objects의 그래프에 타이틀과 축레이블을 표기하는 방법 --> update_layout 사용()
fig.update_layout(title='Stoke Price of XXX (OHLC, $)',
                 yaxis_title='$',
                 )
fig.layout.height = 600 #figure의 크기설정 (너비는 width)
fig.show()

### 이동 평균선 생성(Moving Average)

In [381]:
# 종가를 이용한 MA
ma_3 = df.Close.rolling(3).mean()
ma_10 = df.Close.rolling(10).mean()
ma_30 = df.Close.rolling(30).mean()

In [382]:
df['MA3'] = ma_3
df['MA10'] = ma_10
df['MA30'] = ma_30

In [383]:
display(df) # 날짜평균이므로 앞부분 NaN 발생

,High,Low,Open,Close,Volume,MA3,MA10,MA30
Date,,,,,,,,
2020-01-02,282.8,280.6,282.8,280.6,1.589256e+06,NaN,NaN,NaN
2020-01-03,282.0,278.8,282.0,278.8,1.509149e+06,NaN,NaN,NaN
2020-01-06,281.3,280.8,281.3,280.8,1.412961e+06,280.066667,NaN,NaN
2020-01-07,280.6,280.5,280.6,280.5,1.402866e+06,280.033333,NaN,NaN
2020-01-08,280.6,279.9,280.6,279.9,1.301200e+06,280.400000,NaN,NaN
...,...,...,...,...,...,...,...,...
2021-12-24,314.4,313.4,314.4,313.4,1.466620e+06,314.800000,315.96,315.643333
2021-12-27,315.6,314.9,314.9,315.6,1.366349e+06,314.866667,315.86,315.720000
2021-12-28,314.3,314.1,314.1,314.3,1.315531e+06,314.433333,315.66,315.726667


캔들스틱 차트에 이동편균선을 그려보겠습니다. 우선 Figure 객체에 넣어줄 라인 그래프를 만들겠습니다.

In [384]:
# moving average(이동 평균을 라인그래프로 표현)
ma3 = go.Scatter(
    x=df.index,
    y=df.MA3,
    line =dict(color = 'black', width=0.5),
    name='MA(3)'
    )
ma10 = go.Scatter(
    x=df.index,
    y=df.MA10,
    line =dict(color = 'red', width=0.8),
    name='MA(10)'
    )
ma30 = go.Scatter(
    x=df.index,
    y=df.MA30,
    line =dict(color = 'green', width=1),
    name='MA(30)'
    )

**Figure** 객체의 **add_traces**메서드를 이용하면 쉽게 이동평균 라인을 넣어줄 수 있습니다.

In [385]:
# 기존 fig 객체에 ma3 추가
fig.add_traces(ma3)
fig.show()

In [386]:
fig.add_traces(ma10)
fig.add_traces(ma30)
fig.show()

### Bollinger Bands 추가
볼린저 밴드는 주가 변동이 표준정규분포를 따른다고 가정하고, 주가의 추세를 따라 위아래로 폭이 같이 움직이는 밴드를 만들어 주가를 그 밴드 기준선으로 판단하고자 만들어진 지표입니다.

볼린저 밴드를 만드는 다양한 방법이 있지만, 원리는 동일하며 생각보다 간단합니다.
   1. 기준선을 구해야 합니다. 본 실습에선 이미 계산해놓은 **ma30**(이동평균 30일)로 설정하겠습니다.
   2. 기준선에 대응하는 표준편차를 구해야 합니다. --> df.Close.rolling(30).std
   3. 상한선 = 기준선 + (2*표준편차)
   4. 하한선 = 기준선 - (2*표준편차)

In [387]:
# 표준편차 구하기
std_30 = df.Close.rolling(30).std()

In [388]:
df['STD30'] = std_30

In [389]:
# Bollinger Bands 그리기

# 상한성

upper_bound = go.Scatter(
                x=df.index, # x축 : 날짜
                y = df.MA30 + (2 * df.STD30), # y축 : 상한선 값
                line = dict(dash='dash', color='gray', width=1), # 회색 대시라인 굵기 1
                name='upper bounds' # 선 이름
                )

lower_bound = go.Scatter(
                x=df.index, # x축 : 날짜
                y = df.MA30 - (2 * df.STD30), # y축 : 상한선 값
                line = dict(dash='dash', color='gray', width=1), # 회색 대시라인 굵기 1
                fill = 'tonexty', # 상한선 하한선 사이 면적 채우기
                opacity = 0.5, # 투명도 조절
                name='lower bounds' # 선 이름
                )

In [390]:
# 캔들 스틱에 상한선, 하한선 추가
fig.add_traces([upper_bound,lower_bound])
fig.show()

### 바그래프 : 거래량 추가

In [391]:
# 거래량을 나타내는 trading_volume의 바 플롯
trading_volume = go.Bar(x=df.index, y=df.Volume)

In [392]:
import plotly.subplots as ms #거래량 바그래프 추가를 위해 plotly의 subplot 패키지 로딩

In [393]:
# (2x1) Axes를 갖고 있는 figure 생성

fig = ms.make_subplots(rows=2,
                      cols=1,
                      shared_xaxes=True, # x축 값 공유
                      # vertical_spacing=0.02,
                      subplot_titles = ('Stock Prices of XXX ($)', 'Trading Volumes') # 두개의 플롯에 타이틀 생성 (1번째 플롯, 2번째 플롯)
                      )

In [394]:
fig

In [395]:
# 캔들스틱 그래프 추가 (add_trace를 사용함, add_trace's' 아님)
fig.add_trace(
    go.Candlestick(
        x=df.index, # x축 날짜
        open=df.Open, # 시초가
        high=df.High, # 고가
        low=df.Low, # 저가
        close=df.Close, # 종가
        increasing_line_color='red', # 상승은 빨강
        decreasing_line_color='blue' # 하락은 파랑
    ), row=1, col=1 # 몇번째에 그릴건지
)


In [396]:
# 이동평균선 추가 (1행 1열 그래프 추가)
fig.add_trace(ma3, row=1, col=1)
fig.add_trace(ma10, row=1, col=1)
fig.add_trace(ma30, row=1, col=1)
fig.show()

In [397]:
# 볼린저 라인추가
fig.add_trace(upper_bound, row=1,col=1) #상한선
fig.add_trace(lower_bound, row=1, col=1) #하한선
fig.show()

In [398]:
# 거래량 추가
# 거래량 2행 1열 - 즉 두번째 그래프 추가
fig.add_trace(trading_volume, row=2, col=1)
fig.show()

In [401]:
# figure 레이아웃 설정
fig.update_layout(
    yaxis1_title='Stock Price ($)', # 첫번째 캔들 스틱에 대한 y축 레이블
    yaxis2_title='Trading Volumes',
    xaxis2_title='Periods',
    xaxis1_rangeslider_visible=False,
    xaxis2_rangeslider_visible=True,
)

fig.show()

ploty 차트를 png 형식으로 받고싶다면 단순하게 그래프 우상단에 위치한 카메라 모양 버튼을 누르면 저장할 수 있습니다.

하지만 동적인 요소는 저장되지 않습니다.
동적인 요소 까지 녹여내고 싶다면, html 파일로 저장을 해야합니다.
아래의 커맨드를 입력하면 쉽게 저장할 수 있습니다.

In [403]:
fig.write_html('./interactive_candel_stick.html')